In [ ]:
from huggingface_hub import notebook_login
notebook_login()

Define a function to run inference on a given model on a set of pre-defined prompts and return an array with the responses.

In [ ]:
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes

In [ ]:
# List of prompts
prompts = [
"""Generator is an expert SPARQL query generator.
For each question that the user supplies, the generator will convert it into a valid SPARQL query that can be used to answer the question.
Human: "Where is the Dorset county located?"
Generator:""",
"""Generator creates valid SPARQL queries.The user will provide a question and the generator will convert it into an equivalent SPARQL query that answers the user's question.
Human: "What is the population of Aegina ?"
Generator: """,
"""Generator is an expert SPARQL query generator. For each question that the user supplies, the generator will convert it into a valid SPARQL query that can be used to answer the question. The query will be based on the YAGO knowledge graph.
Human: "Where is the Dorset county located?"
Generator:""",
"""Generator is an expert SPARQL query generator. For each question that the user supplies, the generator will convert it into a valid SPARQL query that can be used to answer the question. The query will be based on the DBpedia knowledge graph.
Human: "What is the river whose mouth is in deadsea?"
Generator:""",
"""Generator is an expert SPARQL query generator. For each question that the user supplies, the generator will convert it into a valid SPARQL query that can be used to answer the question. The query will be based on the Wikidata knowledge graph.
Human: "Was Hans Ertl a screenwriter?"
Generator:""",
"""Generator is an expert SPARQL query generator.
For each question that the user supplies, the generator will convert it into a valid SPARQL query that can be used to answer the question.
Human: "Where is Oxfordshire located?"
Generator: "SELECT ?WKT WHERE { yago:Oxfordshire geo:hasGeometry ?o. ?o geo:asWKT ?WKT. }"
Human: "What is the total area of County Galway?"
Generator:""",
"""Generator is an expert SPARQL query generator. For each question that the user supplies, the generator will convert it into a valid SPARQL query that can be used to answer the question. The query will be based on the YAGO knowledge graph.
Human: "What is the total area of Glengarra Wood forest?"
Generator: "select distinct (strdf:area(?geoWKT) as ?area) where { <http://yago-knowledge.org/resource/geoentity_Glengarra_Wood_3300941> geo:hasGeometry ?o. ?o geo:asWKT ?geoWKT. }"
Human: "What is the population of Piraeus?"
Generator:""",
"""Generator is an expert SPARQL query generator. For each question that the user supplies, the generator will convert it into a valid SPARQL query that can be used to answer the question. The query will be based on the DBpedia knowledge graph.
Human: "What is the river whose mouth is in deadsea?"
Generator: "SELECT DISTINCT ?uri WHERE {?uri <http://dbpedia.org/ontology/riverMouth> <http://dbpedia.org/resource/Dead_Sea>  . ?uri <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/River>}"
Human: "What is the region of Tom Perriello ?"
Generator:""",
"""Generator is an expert SPARQL query generator. For each question that the user supplies, the generator will convert it into a valid SPARQL query that can be used to answer the question. The query will be based on the Wikidata knowledge graph.
Human: "Was Hans Ertl a screenwriter?"
Generator: "ASK WHERE {\nwd:Q103013 wdt:P106 wd:Q69423232}"
Human: "Who were Jean-François Champollion's parents?"
Generator:""",
"""Generator is an expert SPARQL query generator.
For each question that the user supplies, the generator will convert it into a valid SPARQL query that can be used to answer the question.
Human: "Where is Oxfordshire located?"
Generator: "SELECT ?WKT WHERE { yago:Oxfordshire geo:hasGeometry ?o. ?o geo:asWKT ?WKT. }"
Human: "What is Dublin's administrative type?"
Generator: "select ?e where { yago:Dublin rdf:type ?e }"
Human: "What population does Icaria have?"
Generator: "SELECT ?population WHERE{ yago:Icaria  y2geoo:hasGAG_Population ?population . }"
Human: "Where is Scotland located?"
Generator:""",
"""Generator is an expert SPARQL query generator.
For each question that the user supplies, the generator will convert it into a valid SPARQL query that can be used to answer the question.
Human: "Which localities are located east of forests in County Wicklow?"
Generator: "SELECT DISTINCT ?a WHERE { yago:County_Wicklow geo:hasGeometry ?o . ?o geo:asWKT ?geoWKT . ?a rdf:type y2geoo:OSM_locality; geo:hasGeometry ?o1 . ?o1 geo:asWKT ?geoWKT1 . ?b rdf:type y2geoo:OSM_forest; geo:hasGeometry ?o2 . ?o2 geo:asWKT ?geoWKT2 . FILTER (strdf:within(?geoWKT1, ?geoWKT) &&  strdf:within(?geoWKT2, ?geoWKT) &&  strdf:right(?geoWKT1, ?geoWKT2)) }"
Human: "Is there a stream located east of a lake in Corfu?"
Generator: "ASK { yago:Corfu geo:hasGeometry ?o2 . ?o2 geo:asWKT ?xWKT2 . ?x2 rdf:type y2geoo:OSM_stream . ?x2 geo:hasGeometry ?x2Geom. ?x2Geom geo:asWKT ?iWKT2. ?x1 rdf:type y2geoo:OSM_lake . ?x1 geo:hasGeometry ?x1Geom. ?x1Geom geo:asWKT ?iWKT1. FILTER(geof:sfWithin(?iWKT1, ?xWKT2) && geof:sfWithin(?iWKT2,?xWKT2) && strdf:right(?iWKT2,?iWKT1)) }"
Human: "Which region of Greece has the most inhabitants?"
Generator: "SELECT DISTINCT ?region WHERE { ?region rdf:type y2geoo:GAG_Region . ?region y2geoo:hasGAG_Population ?population } ORDER BY DESC (?population) LIMIT 1"
Human: "Which county in the British Isles is the smallest by area?"
Generator:""",
"""Generator is an expert SPARQL query generator. For each question that the user supplies, the generator will convert it into a valid SPARQL query that can be used to answer the question. The query will be based on the YAGO knowledge graph.
Human: "Which are the 2 newest bridges of Ireland?"
Generator: "SELECT DISTINCT ?bridge  WHERE {  ?type rdfs:subClassOf+ yago:wordnet_bridge_102898711. ?bridge a ?type .?bridge yago:isLocatedIn+ yago:California.?bridge yago:wasCreatedOnDate ?date.} ORDER BY DESC(?date)  LIMIT 2"
Human: "Is there a stream located east of a lake in Corfu?"
Generator: "ASK { yago:Corfu geo:hasGeometry ?o2 . ?o2 geo:asWKT ?xWKT2 . ?x2 rdf:type y2geoo:OSM_stream . ?x2 geo:hasGeometry ?x2Geom. ?x2Geom geo:asWKT ?iWKT2. ?x1 rdf:type y2geoo:OSM_lake . ?x1 geo:hasGeometry ?x1Geom. ?x1Geom geo:asWKT ?iWKT1. FILTER(geof:sfWithin(?iWKT1, ?xWKT2) && geof:sfWithin(?iWKT2,?xWKT2) && strdf:right(?iWKT2,?iWKT1)) }"
Human: "Which region of Greece has the most inhabitants?"
Generator: "SELECT DISTINCT ?region WHERE { ?region rdf:type y2geoo:GAG_Region . ?region y2geoo:hasGAG_Population ?population } ORDER BY DESC (?population) LIMIT 1"
Human: "Is Leitrim the least populated county in the Republic of Ireland?"
Generator:""",
"""Generator is an expert SPARQL query generator. For each question that the user supplies, the generator will convert it into a valid SPARQL query that can be used to answer the question. The query will be based on the DBpedia knowledge graph.
Human: "What is the river whose mouth is in deadsea?"
Generator: "SELECT DISTINCT ?uri WHERE {?uri <http://dbpedia.org/ontology/riverMouth> <http://dbpedia.org/resource/Dead_Sea>  . ?uri <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://dbpedia.org/ontology/River>}"
Human: "What is the region of Tom Perriello ?"
Generator: "SELECT DISTINCT ?uri WHERE { <http://dbpedia.org/resource/Tom_Perriello> <http://dbpedia.org/ontology/region> ?uri }"
Human: "How many people live in Wilton, Connecticut?"
Generator: "SELECT DISTINCT COUNT(?uri) WHERE {?uri <http://dbpedia.org/property/residence> <http://dbpedia.org/resource/Wilton,_Connecticut>  . }"
Human: "In which part of the world can i find Xynisteri and Mavro?"
Generator:""",
"""Generator is an expert SPARQL query generator. For each question that the user supplies, the generator will convert it into a valid SPARQL query that can be used to answer the question. The query will be based on the Wikidata knowledge graph.
Human: "Was Hans Ertl a screenwriter?"
Generator: "ASK WHERE {\nwd:Q103013 wdt:P106 wd:Q69423232}"
Human: "Who were Jean-François Champollion's parents?"
Generator: "SELECT DISTINCT ?x0 WHERE {\n?x0 wdt:P40|wdt:P355 wd:Q260 \n}"
Human: "What did Andrei Tarkovsky edit?"
Generator: "SELECT DISTINCT ?x0 WHERE {\n?x0 wdt:P1040 wd:Q853 \n}"
Human: "Was Andrei Tarkovsky a screenwriter?"
Generator:"""]

In [5]:
import torch

def run_inference(model, tokenizer, model_name):
    results = []
    
    print(f"Using model: {model_name}")
    
    # Loop through each prompt
    for prompt in prompts:
        if tokenizer == None:
            # Generate output
            with torch.no_grad():
                outputs = model(prompt)
            
            # Decode and print output
            print("Prompt:", prompt)
            print("Generated text:" + outputs + "\n")
            results.append("Generated text:" + outputs)
        else:
            # Move model to GPU
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model.to(device)
            model.eval()  # Set model to evaluation mode
            
            # Tokenize prompt
            inputs = tokenizer(prompt, return_tensors="pt").to(device)
            
            # Generate output
            with torch.no_grad():
                outputs = model.generate(**inputs, 
                                 max_length=500,  # Set a maximum length for generated text
                                 #do_sample=True,  # Enable sampling
                                 #top_k=7,        # Top-k sampling
                                 #top_p=0.1,      # Top-p sampling (nucleus sampling)
                                 #num_return_sequences=1,
                                 #repetition_penalty=1, # No penalty for instruction tuned models.
                                 repetition_penalty=1.2, # Penalty on repeating tokens.
                                 eos_token_id=tokenizer.eos_token_id,  # Specify EOS token ID
                                 pad_token_id=tokenizer.pad_token_id  # Specify PAD token ID
                                )
        
            # Extract generated text
            generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Remove the prompt text
            prompt_length = len(prompt)
            generated_text = generated_text[prompt_length:]

            # Decode and print output
            print("Prompt:", prompt)
            print("Generated text:" + generated_text + "\n")
            results.append("Generated text:" + generated_text)
    
    # Clear model from RAM
    del model
    torch.cuda.empty_cache()
    
    return results

## Now run inference for each of the following models:

* ### meta-llama/Llama-2-7b-hf

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
 
run_inference(model, tokenizer, "llama-2-7b-hf")

* ### ekshat/Llama-2-7b-chat-finetune-for-text2sql

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel

model = AutoModelForCausalLM.from_pretrained("ekshat/Llama-2-7b-chat-finetune-for-text2sql", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("ekshat/Llama-2-7b-chat-finetune-for-text2sql")

run_inference(model, tokenizer, "ekshat/Llama-2-7b-chat-finetune-for-text2sql")

* ### TheBloke/Llama-2-13B-GGUF

In [ ]:
!pip install ctransformers

In [ ]:
from ctransformers import AutoModelForCausalLM

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
model = AutoModelForCausalLM.from_pretrained("TheBloke/Llama-2-13B-GGUF", model_file="llama-2-13b.Q6_K.gguf", model_type="llama", gpu_layers=50)

run_inference(model, tokenizer=None, model_name="TheBloke/Llama-2-13B-GGUF")

* ### TheBloke/CodeLlama-13B-GGUF

In [ ]:
from ctransformers import AutoModelForCausalLM

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
model = AutoModelForCausalLM.from_pretrained("TheBloke/CodeLlama-13B-GGUF", model_file="codellama-13b.Q6_K.gguf", model_type="llama", gpu_layers=50)

run_inference(model, tokenizer=None, model_name="TheBloke/CodeLlama-2-13B-GGUF")

* ### Mistral-7b-v0.2

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel

model = AutoModelForCausalLM.from_pretrained("alpindale/Mistral-7B-v0.2-hf", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("alpindale/Mistral-7B-v0.2-hf")

run_inference(model, tokenizer, "alpindale/Mistral-7B-v0.2-hf")

* ### Mistral-7b-Instruct-v0.2

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

run_inference(model, tokenizer, "mistralai/Mistral-7B-Instruct-v0.2")

* ### Dolphin-2.8-mistral-7b-v0.2

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel

model = AutoModelForCausalLM.from_pretrained("cognitivecomputations/dolphin-2.8-mistral-7b-v02", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("cognitivecomputations/dolphin-2.8-mistral-7b-v02")

run_inference(model, tokenizer, "cognitivecomputations/dolphin-2.8-mistral-7b-v02")

* ### Hermes-2-Pro-Mistral-7b

In [7]:
import torch

def t5_inference(model, tokenizer, model_name):
    results = []
    
    print(f"Using model: {model_name}")
    
    # Loop through each prompt
    for prompt in prompts:
        # Move model to GPU
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        model.eval()  # Set model to evaluation mode
            
        input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
        outputs = model.generate(input_ids, min_length=500, max_length=1000, early_stopping=False, 
                                 repetition_penalty=1.2, # Penalty on repeating tokens.
                                 eos_token_id=tokenizer.eos_token_id,  # Specify EOS token ID
                                 pad_token_id=tokenizer.pad_token_id  # Specify PAD token ID
                                )
                                
        # Extract generated text
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Decode and print output
        print("Prompt:", prompt)
        print("Generated text:" + generated_text + "\n")
        results.append("Generated text:" + generated_text)
    
    # Clear model from RAM
    del model
    torch.cuda.empty_cache()
    
    return results

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel

model = AutoModelForCausalLM.from_pretrained("NousResearch/Hermes-2-Pro-Mistral-7B", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Hermes-2-Pro-Mistral-7B")

run_inference(model, tokenizer, "NousResearch/Hermes-2-Pro-Mistral-7B")

* ### Google T5-3b

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")

t5_inference(model, tokenizer, "T5-3b")

* ### Google T5v1.1-XL

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/t5-v1_1-xl")
model = AutoModelForSeq2SeqLM.from_pretrained("google/t5-v1_1-xl")

t5_inference(model, tokenizer, "T5v1.1-xl")

* ### Falcon-7b-instruct

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("tiiuae/falcon-7b-instruct", trust_remote_code=True, torch_dtype=torch.float16)

run_inference(model, tokenizer, "Falcon-7b-Instruct")

* ### Llama-3-8b

In [5]:
import torch

def Quantized_Inference(model, tokenizer, model_name):
    results = []
    
    print(f"Using model: {model_name}")
    
    # Loop through each prompt
    for prompt in prompts:
        # Move model to GPU
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.eval()  # Set model to evaluation mode
            
        # Tokenize prompt
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
            
        # Generate output
        with torch.no_grad():
            outputs = model.generate(**inputs, 
                                 max_length=350,  # Set a maximum length for generated text
                                 #do_sample=True,  # Enable sampling
                                 #top_k=7,        # Top-k sampling
                                 #top_p=0.1,      # Top-p sampling (nucleus sampling)
                                 #num_return_sequences=1,
                                 repetition_penalty=1.2, # Penalty on repeating tokens.
                                 eos_token_id=tokenizer.eos_token_id,  # Specify EOS token ID
                                 pad_token_id=tokenizer.pad_token_id  # Specify PAD token ID
                                )
        
        # Extract generated text
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Remove the prompt text
        prompt_length = len(prompt)
        generated_text = generated_text[prompt_length:]

        # Decode and print output
        print("Prompt:", prompt)
        print("Generated text:" + generated_text + "\n")
        results.append("Generated text:" + generated_text)
    
    # Clear model from RAM
    del model
    torch.cuda.empty_cache()
    
    return results

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B", device_map="auto", load_in_8bit=True)

print(model.get_memory_footprint())

Quantized_Inference(model, tokenizer, "Llama-3-8b")

* ### Llama-3-8b-Instruct

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", device_map="auto", load_in_8bit=True)

Quantized_Inference(model, tokenizer, "Llama-3-8b_Instruct")

* ### Google Gemma 2b base

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b", device_map="auto", torch_dtype=torch.float32)

run_inference(model, tokenizer, "Gemma-2b")

* ### Google Gemma 2b Instruct

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto", torch_dtype=torch.float32)

run_inference(model, tokenizer, "Gemma-2b-it")